# Meteoroid Glitch Injector

The contents of this notebook is made to guide the user to (1) generate files from NASA's Meteoroid Engineering Model and (2) connect these files to LISANode simulator.

In [1]:
# Import libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load relevant functions

In [3]:
def GrunModel(m):
    C     = np.array([2.2e3, 15, 1.3e-9, 1e11, 1e27, 1.3e-16, 1e6])
    g     = np.array([0.306, -4.38, 2, 4, -0.36, 2, -0.85])

    G = ((C[0]*(m**g[0])) + C[1])**g[1] + \
           C[2]*(m + C[3]*(m**g[2]) + C[4]*(m**g[3]))**g[4] + \
           C[5]*(m + C[6]*(m**g[5]))**g[6]
    
    return G

## 1. Load MEM data

In [8]:
files               = ['LISA_sc1_04_18_2023', 'LISA_sc2_04_30_2023', 'LISA_sc3_04_30_2023']
densities           = ['HiDensity','LoDensity']

numflux = 1652
numv    = 80

momd = {}
mommapd = {}

for nindex, name in enumerate(files):
    for dindex, density in enumerate(densities):
        
        ##### PREPROCESSING #####

        # We first need to make sure we are returning the correct velocity bin values
        # Note, when we call cast the raw data to a pandas dataframe, the column names are offset by one element, so
        # we take caution in initializing the values that correspond to the flux values.
        velocity_bins = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True) 

        # Here we simply just read the table using pandas
        iglooavg = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/{density}/igloo_avg.txt",sep="\s+", header = 7)

        # We take the matrix of flux values dependent on velocity and cast it to a numpy array with appropriate dimensions for processing
        Fd = np.array([iglooavg[str(velocity_bins[i])] for i in range(numv)]).T

        # Since each raw flux value is per solid angle interval, we need dPhi and dTheta to correct for that interval
        dPhi   = (iglooavg['PHI1'] - iglooavg['J']) * (np.pi/180);
        dTheta = (iglooavg['THETA1'] - iglooavg['PHI2']) * (np.pi/180);

        # Construct preprocessed flux matrix using a vectorized for loop
        Flux = np.array([[.5 * Fd[i, j] * np.cos(iglooavg['THETA2'].to_numpy()[i] * (np.pi/180)) * dPhi[i] * dTheta[i] for j in range(numv)] for i in range(numflux)])

        ##### GRUN MODEL PROCESS #####
        # Constants
        numbins = 10
        vbins = np.linspace(0.5, 79.5, numv)
        momentumbins = np.logspace(-6, 3, numbins)
        mass = np.logspace(-6, 1, numbins)

        # Computing Grun Model flux 
        # Output: Flux_Grun(mass)
        G = GrunModel(mass) * (365.25 * 24 * 3600)
        dgrun = np.array([G[k] - G[k+1] for k in range(numbins - 1)])
        dgrun /= np.sum(dgrun)
        
        # Construct Momentum Matrix w/ MEM velocity bins and mass array
        momentum_mat = vbins.reshape(-1,1)@mass.reshape(1,-1)
        
        # Expanding flux array with an extra dimension: m (mass)
        # Output: F(phi, theta, velocity, mass)
        fluxM = np.array([[[Flux[ii, jj] * dgrun[kk] for kk in range(numbins-1)] for jj in range(numv)] for ii in range(numflux)])
        
        # Bin the expanded flux values into respective momentum bins
        # Output: F_total_1d(momentum)
        F_total_1d = np.zeros(numbins)
        for l in range(numbins - 1):
            for i in range(numflux):
                for j in range(numv):
                    for k in range(numbins - 1):
                        if l == 0:
                                if (momentum_mat[j, k] <= momentumbins[l]):
                                    F_total_1d[l]     += fluxM[i, j, k]   
                        elif l == (numbins - 1):
                                if (momentumbins[l] <= momentum_mat[j, k]):
                                    F_total_1d[l]     += fluxM[i, j, k]    
                        else: 
                            if (momentumbins[l] <= momentum_mat[j, k] < momentumbins[l + 1]):
                                    F_total_1d[l]     += fluxM[i, j, k] 

        # Bin the expanded flux values into respective momentum and sky position bins
        # Output: F_total_2d(sky position, momentum)          
        F_total_2d = np.zeros([numflux, numbins])
        for l in range(numbins - 1):
            for i in range(numflux):
                for j in range(numv):
                    for k in range(numbins - 1):
                        if l == 0:
                                if (momentum_mat[j, k] <= momentumbins[l]):
                                    F_total_2d[i, l]     += fluxM[i, j, k]   
                        elif l == (numbins - 1):
                                if (momentumbins[l] <= momentum_mat[j, k]):
                                    F_total_2d[i, l]     += fluxM[i, j, k]
                        else: 
                            if (momentumbins[l] <= momentum_mat[j, k] < momentumbins[l + 1]):
                                    F_total_2d[i, l]     += fluxM[i, j, k]

        # Store data into respective dictionaries
        momd[f'{spacecraft_names[nindex]}_{density}'] = F_total_1d
        mommapd[f'{spacecraft_names[nindex]}_{density}'] = F_total_2d

In [9]:
# Compute total flux across full constellation
F_total_1d = (momd['Spacecraft 1_HiDensity'] + momd['Spacecraft 1_LoDensity'])  + \
             (momd['Spacecraft 2_HiDensity'] + momd['Spacecraft 2_LoDensity'])  + \
             (momd['Spacecraft 3_HiDensity'] + momd['Spacecraft 3_LoDensity']) 

# Compute total flux across full constellation restrict summation to 2d flux map
F_total_2d = (mommapd['Spacecraft 1_HiDensity'] + mommapd['Spacecraft 1_LoDensity'])  + \
             (mommapd['Spacecraft 2_HiDensity'] + mommapd['Spacecraft 2_LoDensity'])  + \
             (mommapd['Spacecraft 3_HiDensity'] + mommapd['Spacecraft 3_LoDensity']) 

In [109]:
from scipy.stats import poisson

p_total_1d = poisson.rvs(F_total_1d) * 84
p_total_2d = poisson.rvs(F_total_2d) * 84 
print(np.where(p_total_1d==0))
print(np.where(p_total_1d!=0))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),)
(array([], dtype=int64),)


In [48]:
F_total_1d

array([1.02093911e-07, 6.26461559e-02, 1.00800202e-02, 8.59027996e-04,
       3.84934919e-05, 1.88463627e-06, 1.39626828e-07, 5.38896756e-09,
       3.89982249e-11, 0.00000000e+00])

- The issue I am running into now is the fact that we have too little events happening and trying to feed those back in the simulator is going to be a pain
- Let's see if we can try to make a 3d model of a cylinder and extract an impact angle and location on cylinder using sky position values from above.

In [ ]:
# Plotting a 3d cylinder on a 2d projection